In [3]:
import os

import pandas as pd
import numpy as np

from sklearn.model_selection import KFold, train_test_split

import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn as nn 
import torch.multiprocessing
from torchvision import transforms
#!pip install einops
from einops import rearrange
torch.multiprocessing.set_sharing_strategy('file_system')

from tqdm import tqdm

from torchvision import transforms
from PIL import Image, ImageSequence
Image.MAX_IMAGE_PIXELS = None

from botocore import UNSIGNED
from botocore.config import Config
import boto3
import io

import math
from functools import partial

import cv2

#!apt -y install --fix-missing libvips libvips-dev
#!pip install pyvips
#import pyvips

In [4]:
train_labels = pd.read_csv('../data/train_labels.csv')
train_metadata = pd.read_csv('../data/train_metadata.csv')

train = train_metadata.merge(train_labels, on='filename', how='inner')

def process_age(age_str):
    age_str = age_str.replace('[', '').replace(']', '').split(':')
    return (int(age_str[0]) + int(age_str[1])) / 2

body_map = ['thigh', 'trunc', 'face', 'forearm', 'arm', 'leg', 'hand', 'foot', 'sole', 'finger', 'neck', 'toe', 'seat', 'scalp', 'nail','lower limb/hip', 'hand/foot/nail', 'head/neck', 'upper limb/shoulder', 'other']
def process_body(body):
    if body in ['thigh', 'trunc', 'face', 'forearm', 'arm', 'leg', 'hand', 'foot', 'sole', 'finger', 'neck', 'toe', 'seat', 'scalp', 'nail']:
        return body_map.index(body)
    else:
        return body_map.index('other')
   
mel_map = {'other': 0, 'YES': 1, 'NO': 2}

train.age = train.age.apply(lambda x: process_age(x)).astype(int)
train.body_site = train.body_site.replace('trunk', 'trunc') .fillna('other').apply(process_body).astype(int)
train.melanoma_history = train.melanoma_history.fillna('other').apply(lambda x: mel_map[x]).astype(int)

print(train.shape)

(1342, 14)


In [5]:
class VisioMel_Dataset(Dataset):
    
    def __init__(self, data):
        
        data = data.reset_index(drop=True)
        
        self.filenames, self.y = data['filename'], data['relapse']
        
        self.age = data['age'] / 100
        self.sex = data['sex'] - 1
        self.body_site = data['body_site']
        self.melanoma_history = data['melanoma_history']
        
        #self.mean, self.std = (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)
        self.eval_t = transforms.Compose([transforms.ToTensor(), 
                                          #transforms.Normalize(mean = self.mean, std = self.std)#,
                                          #transforms.Resize((65536,65536))
                                        ])
        # 32768 65536
        self.labels = torch.tensor(self.y.values, dtype = torch.float32)
        
        # To tensor labels
        self.age = torch.tensor(self.age.values, dtype = torch.float32)
        self.sex = torch.tensor(self.sex.values, dtype = torch.long)
        self.body_site = torch.tensor(self.body_site.values, dtype = torch.long)
        self.melanoma_history= torch.tensor(self.melanoma_history.values, dtype = torch.long)
        
        #self.s3 = boto3.resource('s3',config=Config(signature_version=UNSIGNED))
        #self.bucket = self.s3.Bucket('drivendata-competition-visiomel-public-eu')
        
        print(f'{self.labels.shape}')
    
    def __len__(self):
        return self.labels.shape[0]
    
    def __getitem__(self, index):
        #image = Image.open('../data/images/'+self.filenames[index])
        #image = pyvips.Image.new_from_file('../data/2/'+self.filenames[index], page=2).numpy()
        #print(image.shape)
        #patches = sk_image.extract_patches_2d(image, (4096, 4096))
        #print(patches.shape)
        #object = self.bucket.Object('images/'+self.filenames[index])
        #img_stream = io.BytesIO()
        #object.download_fileobj(img_stream)
        #self.bucket.download_file('images/'+self.filenames[index], f'{self.filenames[index]}.tif')
        #page = pyvips.Image.new_from_file(f'{self.filenames[index]}.tif', page=4).numpy()
        #page = cv2.resize(page, dsize=(4096 * 4, 4096 * 4), interpolation=cv2.INTER_CUBIC)
        #os.remove(f'{self.filenames[index]}.tif')
        #image = Image.open(img_stream)
        #it = ImageSequence.Iterator(image)
        #page = it[3]
        #h, w = page.size
        #h = h if h >= 4096 else 4096
        #w = w if w >= 4096 else 4096
        #page = cv2.resize(np.array(page), dsize=(h, w), interpolation=cv2.INTER_CUBIC)#.numpy()
        #page = cv2.imread(img_stream)
        #image = torch.Tensor(page, dtype=torch.uint8)
        page = np.load('D:/VictorCallejas/images/' + f'{self.filenames[index]}.npy')
        image = self.eval_t(page).unsqueeze(0)
        patches = image.unfold(2, 4096, 4096).unfold(3,4096, 4096).squeeze(0)
        patches = patches.reshape(-1, 3, 4096, 4096)
        return patches, self.age[index], self.sex[index], self.body_site[index], self.melanoma_history[index], self.labels[index]

In [6]:
train, test = train_test_split(train, test_size=0.2, random_state=42)

train_dataset = VisioMel_Dataset(train)
valid_dataset = VisioMel_Dataset(test)

train_dataloader = DataLoader(
                train_dataset,  
                sampler = RandomSampler(train_dataset),
                batch_size = 1,
                num_workers = 0
            )

valid_dataloader = DataLoader(
            valid_dataset,  
            sampler = SequentialSampler(valid_dataset),
            batch_size = 1,
            num_workers = 0
        )


torch.Size([1073])
torch.Size([269])


In [7]:
def _no_grad_trunc_normal_(tensor, mean, std, a, b):
    # Cut & paste from PyTorch official master until it's in a few official releases - RW
    # Method based on https://people.sc.fsu.edu/~jburkardt/presentations/truncated_normal.pdf
    def norm_cdf(x):
        # Computes standard normal cumulative distribution function
        return (1. + math.erf(x / math.sqrt(2.))) / 2.

    if (mean < a - 2 * std) or (mean > b + 2 * std):
        warnings.warn("mean is more than 2 std from [a, b] in nn.init.trunc_normal_. "
                      "The distribution of values may be incorrect.",
                      stacklevel=2)

    with torch.no_grad():
        # Values are generated by using a truncated uniform distribution and
        # then using the inverse CDF for the normal distribution.
        # Get upper and lower cdf values
        l = norm_cdf((a - mean) / std)
        u = norm_cdf((b - mean) / std)

        # Uniformly fill tensor with values from [l, u], then translate to
        # [2l-1, 2u-1].
        tensor.uniform_(2 * l - 1, 2 * u - 1)

        # Use inverse cdf transform for normal distribution to get truncated
        # standard normal
        tensor.erfinv_()

        # Transform to proper mean, std
        tensor.mul_(std * math.sqrt(2.))
        tensor.add_(mean)

        # Clamp to ensure it's in the proper range
        tensor.clamp_(min=a, max=b)
        return tensor


def trunc_normal_(tensor, mean=0., std=1., a=-2., b=2.):
    # type: (Tensor, float, float, float, float) -> Tensor
    return _no_grad_trunc_normal_(tensor, mean, std, a, b)


def drop_path(x, drop_prob: float = 0., training: bool = False):
    if drop_prob == 0. or not training:
        return x
    keep_prob = 1 - drop_prob
    shape = (x.shape[0],) + (1,) * (x.ndim - 1)  # work with diff dim tensors, not just 2D ConvNets
    random_tensor = keep_prob + torch.rand(shape, dtype=x.dtype, device=x.device)
    random_tensor.floor_()  # binarize
    output = x.div(keep_prob) * random_tensor
    return output


class DropPath(nn.Module):
    """Drop paths (Stochastic Depth) per sample  (when applied in main path of residual blocks).
    """
    def __init__(self, drop_prob=None):
        super(DropPath, self).__init__()
        self.drop_prob = drop_prob

    def forward(self, x):
        return drop_path(x, self.drop_prob, self.training)


class Mlp(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = act_layer()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x


class Attention(nn.Module):
    def __init__(self, dim, num_heads=8, qkv_bias=False, qk_scale=None, attn_drop=0., proj_drop=0.):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = qk_scale or head_dim ** -0.5

        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

    def forward(self, x):
        B, N, C = x.shape
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]

        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x, attn


class Block(nn.Module):
    def __init__(self, dim, num_heads, mlp_ratio=4., qkv_bias=False, qk_scale=None, drop=0., attn_drop=0.,
                 drop_path=0., act_layer=nn.GELU, norm_layer=nn.LayerNorm):
        super().__init__()
        self.norm1 = norm_layer(dim)
        self.attn = Attention(
            dim, num_heads=num_heads, qkv_bias=qkv_bias, qk_scale=qk_scale, attn_drop=attn_drop, proj_drop=drop)
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()
        self.norm2 = norm_layer(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = Mlp(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop)

    def forward(self, x, return_attention=False):
        y, attn = self.attn(self.norm1(x))
        if return_attention:
            return attn
        x = x + self.drop_path(y)
        x = x + self.drop_path(self.mlp(self.norm2(x)))
        return x


class PatchEmbed(nn.Module):
    """ Image to Patch Embedding
    """
    def __init__(self, img_size=224, patch_size=16, in_chans=3, embed_dim=768):
        super().__init__()
        num_patches = (img_size // patch_size) * (img_size // patch_size)
        self.img_size = img_size
        self.patch_size = patch_size
        self.num_patches = num_patches

        self.proj = nn.Conv2d(in_chans, embed_dim, kernel_size=patch_size, stride=patch_size)

    def forward(self, x):
        B, C, H, W = x.shape
        x = self.proj(x).flatten(2).transpose(1, 2)
        return x


class VisionTransformer(nn.Module):
    """ Vision Transformer """
    def __init__(self, img_size=[224], patch_size=16, in_chans=3, num_classes=0, embed_dim=768, depth=12,
                 num_heads=12, mlp_ratio=4., qkv_bias=False, qk_scale=None, drop_rate=0., attn_drop_rate=0.,
                 drop_path_rate=0., norm_layer=nn.LayerNorm, **kwargs):
        super().__init__()
        self.num_features = self.embed_dim = embed_dim

        self.patch_embed = PatchEmbed(
            img_size=img_size[0], patch_size=patch_size, in_chans=in_chans, embed_dim=embed_dim)
        num_patches = self.patch_embed.num_patches

        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        self.pos_embed = nn.Parameter(torch.zeros(1, num_patches + 1, embed_dim))
        self.pos_drop = nn.Dropout(p=drop_rate)

        dpr = [x.item() for x in torch.linspace(0, drop_path_rate, depth)]  # stochastic depth decay rule
        self.blocks = nn.ModuleList([
            Block(
                dim=embed_dim, num_heads=num_heads, mlp_ratio=mlp_ratio, qkv_bias=qkv_bias, qk_scale=qk_scale,
                drop=drop_rate, attn_drop=attn_drop_rate, drop_path=dpr[i], norm_layer=norm_layer)
            for i in range(depth)])
        self.norm = norm_layer(embed_dim)

        # Classifier head
        self.head = nn.Linear(embed_dim, num_classes) if num_classes > 0 else nn.Identity()

        trunc_normal_(self.pos_embed, std=.02)
        trunc_normal_(self.cls_token, std=.02)
        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    def interpolate_pos_encoding(self, x, w, h):
        npatch = x.shape[1] - 1
        N = self.pos_embed.shape[1] - 1
        if npatch == N and w == h:
            return self.pos_embed
        class_pos_embed = self.pos_embed[:, 0]
        patch_pos_embed = self.pos_embed[:, 1:]
        dim = x.shape[-1]
        w0 = w // self.patch_embed.patch_size
        h0 = h // self.patch_embed.patch_size
        # we add a small number to avoid floating point error in the interpolation
        # see discussion at https://github.com/facebookresearch/dino/issues/8
        w0, h0 = w0 + 0.1, h0 + 0.1
        patch_pos_embed = nn.functional.interpolate(
            patch_pos_embed.reshape(1, int(math.sqrt(N)), int(math.sqrt(N)), dim).permute(0, 3, 1, 2),
            scale_factor=(w0 / math.sqrt(N), h0 / math.sqrt(N)),
            mode='bicubic',
        )
        assert int(w0) == patch_pos_embed.shape[-2] and int(h0) == patch_pos_embed.shape[-1]
        patch_pos_embed = patch_pos_embed.permute(0, 2, 3, 1).view(1, -1, dim)
        return torch.cat((class_pos_embed.unsqueeze(0), patch_pos_embed), dim=1)

    def prepare_tokens(self, x):
        B, nc, w, h = x.shape
        x = self.patch_embed(x)  # patch linear embedding

        # add the [CLS] token to the embed patch tokens
        cls_tokens = self.cls_token.expand(B, -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)

        # add positional encoding to each token
        x = x + self.interpolate_pos_encoding(x, w, h)

        return self.pos_drop(x)

    def forward(self, x):
        x = self.prepare_tokens(x)
        for blk in self.blocks:
            x = blk(x)
        x = self.norm(x)
        return x[:, 0]

    def get_last_selfattention(self, x):
        x = self.prepare_tokens(x)
        for i, blk in enumerate(self.blocks):
            if i < len(self.blocks) - 1:
                x = blk(x)
            else:
                # return attention of the last block
                return blk(x, return_attention=True)

    def get_intermediate_layers(self, x, n=1):
        x = self.prepare_tokens(x)
        # we return the output tokens from the `n` last blocks
        output = []
        for i, blk in enumerate(self.blocks):
            x = blk(x)
            if len(self.blocks) - i <= n:
                output.append(self.norm(x))
        return output


def vit_tiny(patch_size=16, **kwargs):
    model = VisionTransformer(
        patch_size=patch_size, embed_dim=192, depth=12, num_heads=3, mlp_ratio=4,
        qkv_bias=True, norm_layer=partial(nn.LayerNorm, eps=1e-6), **kwargs)
    return model


def vit_small(patch_size=16, **kwargs):
    model = VisionTransformer(
        patch_size=patch_size, embed_dim=384, depth=12, num_heads=6, mlp_ratio=4,
        qkv_bias=True, norm_layer=partial(nn.LayerNorm, eps=1e-6), **kwargs)
    return model


def vit_base(patch_size=16, **kwargs):
    model = VisionTransformer(
        patch_size=patch_size, embed_dim=768, depth=12, num_heads=12, mlp_ratio=4,
        qkv_bias=True, norm_layer=partial(nn.LayerNorm, eps=1e-6), **kwargs)
    return model


class DINOHead(nn.Module):
    def __init__(self, in_dim, out_dim, use_bn=False, norm_last_layer=True, nlayers=3, hidden_dim=2048, bottleneck_dim=256):
        super().__init__()
        nlayers = max(nlayers, 1)
        if nlayers == 1:
            self.mlp = nn.Linear(in_dim, bottleneck_dim)
        else:
            layers = [nn.Linear(in_dim, hidden_dim)]
            if use_bn:
                layers.append(nn.BatchNorm1d(hidden_dim))
            layers.append(nn.GELU())
            for _ in range(nlayers - 2):
                layers.append(nn.Linear(hidden_dim, hidden_dim))
                if use_bn:
                    layers.append(nn.BatchNorm1d(hidden_dim))
                layers.append(nn.GELU())
            layers.append(nn.Linear(hidden_dim, bottleneck_dim))
            self.mlp = nn.Sequential(*layers)
        self.apply(self._init_weights)
        self.last_layer = nn.utils.weight_norm(nn.Linear(bottleneck_dim, out_dim, bias=False))
        self.last_layer.weight_g.data.fill_(1)
        if norm_last_layer:
            self.last_layer.weight_g.requires_grad = False

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = self.mlp(x)
        x = nn.functional.normalize(x, dim=-1, p=2)
        x = self.last_layer(x)
        return x


def _no_grad_trunc_normal_(tensor, mean, std, a, b):
    # Cut & paste from PyTorch official master until it's in a few official releases - RW
    # Method based on https://people.sc.fsu.edu/~jburkardt/presentations/truncated_normal.pdf
    def norm_cdf(x):
        # Computes standard normal cumulative distribution function
        return (1. + math.erf(x / math.sqrt(2.))) / 2.

    if (mean < a - 2 * std) or (mean > b + 2 * std):
        warnings.warn("mean is more than 2 std from [a, b] in nn.init.trunc_normal_. "
                      "The distribution of values may be incorrect.",
                      stacklevel=2)

    with torch.no_grad():
        # Values are generated by using a truncated uniform distribution and
        # then using the inverse CDF for the normal distribution.
        # Get upper and lower cdf values
        l = norm_cdf((a - mean) / std)
        u = norm_cdf((b - mean) / std)

        # Uniformly fill tensor with values from [l, u], then translate to
        # [2l-1, 2u-1].
        tensor.uniform_(2 * l - 1, 2 * u - 1)

        # Use inverse cdf transform for normal distribution to get truncated
        # standard normal
        tensor.erfinv_()

        # Transform to proper mean, std
        tensor.mul_(std * math.sqrt(2.))
        tensor.add_(mean)

        # Clamp to ensure it's in the proper range
        tensor.clamp_(min=a, max=b)
        return tensor


def trunc_normal_(tensor, mean=0., std=1., a=-2., b=2.):
    # type: (Tensor, float, float, float, float) -> Tensor
    return _no_grad_trunc_normal_(tensor, mean, std, a, b)



def drop_path(x, drop_prob: float = 0., training: bool = False):
    if drop_prob == 0. or not training:
        return x
    keep_prob = 1 - drop_prob
    shape = (x.shape[0],) + (1,) * (x.ndim - 1)  # work with diff dim tensors, not just 2D ConvNets
    random_tensor = keep_prob + torch.rand(shape, dtype=x.dtype, device=x.device)
    random_tensor.floor_()  # binarize
    output = x.div(keep_prob) * random_tensor
    return output


class DropPath(nn.Module):
    """Drop paths (Stochastic Depth) per sample  (when applied in main path of residual blocks).
    """
    def __init__(self, drop_prob=None):
        super(DropPath, self).__init__()
        self.drop_prob = drop_prob

    def forward(self, x):
        return drop_path(x, self.drop_prob, self.training)


class Mlp(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = act_layer()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x


class Attention(nn.Module):
    def __init__(self, dim, num_heads=8, qkv_bias=False, qk_scale=None, attn_drop=0., proj_drop=0.):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = qk_scale or head_dim ** -0.5

        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

    def forward(self, x):
        B, N, C = x.shape
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]

        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x, attn


class Block(nn.Module):
    def __init__(self, dim, num_heads, mlp_ratio=4., qkv_bias=False, qk_scale=None, drop=0., attn_drop=0.,
                 drop_path=0., act_layer=nn.GELU, norm_layer=nn.LayerNorm):
        super().__init__()
        self.norm1 = norm_layer(dim)
        self.attn = Attention(
            dim, num_heads=num_heads, qkv_bias=qkv_bias, qk_scale=qk_scale, attn_drop=attn_drop, proj_drop=drop)
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()
        self.norm2 = norm_layer(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = Mlp(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop)

    def forward(self, x, return_attention=False):
        y, attn = self.attn(self.norm1(x))
        if return_attention:
            return attn
        x = x + self.drop_path(y)
        x = x + self.drop_path(self.mlp(self.norm2(x)))
        return x


class VisionTransformer4K(nn.Module):
    """ Vision Transformer 4K """
    def __init__(self, num_classes=0, img_size=[224], input_embed_dim=384, output_embed_dim = 192,
                 depth=12, num_heads=12, mlp_ratio=4., qkv_bias=False, qk_scale=None, 
                 drop_rate=0., attn_drop_rate=0., drop_path_rate=0., norm_layer=nn.LayerNorm, num_prototypes=64, **kwargs):
        super().__init__()
        embed_dim = output_embed_dim
        self.num_features = self.embed_dim = embed_dim
        self.phi = nn.Sequential(*[nn.Linear(input_embed_dim, output_embed_dim), nn.GELU(), nn.Dropout(p=drop_rate)])
        num_patches = int(img_size[0] // 16)**2
        print("# of Patches:", num_patches)
        
        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        self.pos_embed = nn.Parameter(torch.zeros(1, num_patches + 1, embed_dim))
        self.pos_drop = nn.Dropout(p=drop_rate)

        dpr = [x.item() for x in torch.linspace(0, drop_path_rate, depth)]  # stochastic depth decay rule
        self.blocks = nn.ModuleList([
            Block(
                dim=embed_dim, num_heads=num_heads, mlp_ratio=mlp_ratio, qkv_bias=qkv_bias, qk_scale=qk_scale,
                drop=drop_rate, attn_drop=attn_drop_rate, drop_path=dpr[i], norm_layer=norm_layer)
            for i in range(depth)])
        self.norm = norm_layer(embed_dim)

        # Classifier head
        self.head = nn.Linear(embed_dim, num_classes) if num_classes > 0 else nn.Identity()

        trunc_normal_(self.pos_embed, std=.02)
        trunc_normal_(self.cls_token, std=.02)
        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    def interpolate_pos_encoding(self, x, w, h):
        npatch = x.shape[1] - 1
        N = self.pos_embed.shape[1] - 1
        if npatch == N and w == h:
            return self.pos_embed
        class_pos_embed = self.pos_embed[:, 0]
        patch_pos_embed = self.pos_embed[:, 1:]
        dim = x.shape[-1]
        w0 = w // 1
        h0 = h // 1
        # we add a small number to avoid floating point error in the interpolation
        # see discussion at https://github.com/facebookresearch/dino/issues/8
        w0, h0 = w0 + 0.1, h0 + 0.1
        patch_pos_embed = nn.functional.interpolate(
            patch_pos_embed.reshape(1, int(math.sqrt(N)), int(math.sqrt(N)), dim).permute(0, 3, 1, 2),
            scale_factor=(w0 / math.sqrt(N), h0 / math.sqrt(N)),
            mode='bicubic',
        )
        assert int(w0) == patch_pos_embed.shape[-2] and int(h0) == patch_pos_embed.shape[-1]
        patch_pos_embed = patch_pos_embed.permute(0, 2, 3, 1).view(1, -1, dim)
        return torch.cat((class_pos_embed.unsqueeze(0), patch_pos_embed), dim=1)

    def prepare_tokens(self, x):
        #print('preparing tokens (after crop)', x.shape)
        self.mpp_feature = x
        B, embed_dim, w, h = x.shape
        x = x.flatten(2, 3).transpose(1,2)

        x = self.phi(x)


        # add the [CLS] token to the embed patch tokens
        cls_tokens = self.cls_token.expand(B, -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)

        # add positional encoding to each token
        x = x + self.interpolate_pos_encoding(x, w, h)

        return self.pos_drop(x)

    def forward(self, x):
        x = self.prepare_tokens(x)
        for blk in self.blocks:
            x = blk(x)
        x = self.norm(x)
        return x[:, 0]

    def get_last_selfattention(self, x):
        x = self.prepare_tokens(x)
        for i, blk in enumerate(self.blocks):
            if i < len(self.blocks) - 1:
                x = blk(x)
            else:
                # return attention of the last block
                return blk(x, return_attention=True)

    def get_intermediate_layers(self, x, n=1):
        x = self.prepare_tokens(x)
        # we return the output tokens from the `n` last blocks
        output = []
        for i, blk in enumerate(self.blocks):
            x = blk(x)
            if len(self.blocks) - i <= n:
                output.append(self.norm(x))
        return output
    
def vit4k_xs(patch_size=16, **kwargs):
    model = VisionTransformer4K(
        patch_size=patch_size, input_embed_dim=384, output_embed_dim=192,
        depth=6, num_heads=6, mlp_ratio=4, 
        qkv_bias=True, norm_layer=partial(nn.LayerNorm, eps=1e-6), **kwargs)
    return model

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


In [8]:
def get_vit256(pretrained_weights, arch='vit_small', device=torch.device('cuda')):
    r"""
    Builds ViT-256 Model.
    
    Args:
    - pretrained_weights (str): Path to ViT-256 Model Checkpoint.
    - arch (str): Which model architecture.
    - device (torch): Torch device to save model.
    
    Returns:
    - model256 (torch.nn): Initialized model.
    """
    
    checkpoint_key = 'teacher'
    device = device
    model256 = vit_small(patch_size=16, num_classes=0)
    for p in model256.parameters():
        p.requires_grad = False
    model256.eval()
    model256.to(device)

    if os.path.isfile(pretrained_weights):
        state_dict = torch.load(pretrained_weights, map_location="cpu")
        if checkpoint_key is not None and checkpoint_key in state_dict:
            print(f"Take key {checkpoint_key} in provided checkpoint dict")
            state_dict = state_dict[checkpoint_key]
        # remove `module.` prefix
        state_dict = {k.replace("module.", ""): v for k, v in state_dict.items()}
        # remove `backbone.` prefix induced by multicrop wrapper
        state_dict = {k.replace("backbone.", ""): v for k, v in state_dict.items()}
        msg = model256.load_state_dict(state_dict, strict=False)
        print('Pretrained weights found at {} and loaded with msg: {}'.format(pretrained_weights, msg))
        
    return model256


def get_vit4k(pretrained_weights, arch='vit4k_xs', device=torch.device('cuda')):
    r"""
    Builds ViT-4K Model.
    
    Args:
    - pretrained_weights (str): Path to ViT-4K Model Checkpoint.
    - arch (str): Which model architecture.
    - device (torch): Torch device to save model.
    
    Returns:
    - model256 (torch.nn): Initialized model.
    """
    
    checkpoint_key = 'teacher'
    device = device
    model4k = vit4k_xs(num_classes=0)
    for p in model4k.parameters():
        p.requires_grad = False
    model4k.eval()
    model4k.to(device)

    if os.path.isfile(pretrained_weights):
        state_dict = torch.load(pretrained_weights, map_location="cpu")
        if checkpoint_key is not None and checkpoint_key in state_dict:
            print(f"Take key {checkpoint_key} in provided checkpoint dict")
            state_dict = state_dict[checkpoint_key]
        # remove `module.` prefix
        state_dict = {k.replace("module.", ""): v for k, v in state_dict.items()}
        # remove `backbone.` prefix induced by multicrop wrapper
        state_dict = {k.replace("backbone.", ""): v for k, v in state_dict.items()}
        msg = model4k.load_state_dict(state_dict, strict=False)
        print('Pretrained weights found at {} and loaded with msg: {}'.format(pretrained_weights, msg))
        
    return model4k


def eval_transforms():
	"""
	"""
	mean, std = (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)
	eval_t = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean = mean, std = std)])
	return eval_t


def roll_batch2img(batch: torch.Tensor, w: int, h: int, patch_size=256):
	"""
	Rolls an image tensor batch (batch of [256 x 256] images) into a [W x H] Pil.Image object.
	
	Args:
		batch (torch.Tensor): [B x 3 x 256 x 256] image tensor batch.
		
	Return:
		Image.PIL: [W x H X 3] Image.
	"""
	batch = batch.reshape(w, h, 3, patch_size, patch_size)
	img = rearrange(batch, 'p1 p2 c w h-> c (p1 w) (p2 h)').unsqueeze(dim=0)
	return Image.fromarray(tensorbatch2im(img)[0])


def tensorbatch2im(input_image, imtype=np.uint8):
    r""""
    Converts a Tensor array into a numpy image array.
    
    Args:
        - input_image (torch.Tensor): (B, C, W, H) Torch Tensor.
        - imtype (type): the desired type of the converted numpy array
        
    Returns:
        - image_numpy (np.array): (B, W, H, C) Numpy Array.
    """
    if not isinstance(input_image, np.ndarray):
        image_numpy = input_image.cpu().float().numpy()  # convert it into a numpy array
        #if image_numpy.shape[0] == 1:  # grayscale to RGB
        #    image_numpy = np.tile(image_numpy, (3, 1, 1))
        image_numpy = (np.transpose(image_numpy, (0, 2, 3, 1)) + 1) / 2.0 * 255.0  # post-processing: tranpose and scaling
    else:  # if it is a numpy array, do nothing
        image_numpy = input_image
    return image_numpy.astype(imtype)


In [9]:
class HIPT_4K(torch.nn.Module):
	"""
	HIPT Model (ViT-4K) for encoding non-square images (with [256 x 256] patch tokens), with 
	[256 x 256] patch tokens encoded via ViT-256 using [16 x 16] patch tokens.
	"""
	def __init__(self, 
		model256_path: str = '../Checkpoints/vit256_small_dino.pth',
		model4k_path: str = '../Checkpoints/vit4k_xs_dino.pth',  
		device256=torch.device('cuda'), 
		device4k=torch.device('cuda')):

		super().__init__()
		self.model256 = get_vit256(pretrained_weights=model256_path).to(device256)
		self.model4k = get_vit4k(pretrained_weights=model4k_path).to(device4k)
		self.device256 = device256
		self.device4k = device4k
	
	def forward(self, x):
		"""
		Forward pass of HIPT (given an image tensor x), outputting the [CLS] token from ViT-4K.
		1. x is center-cropped such that the W / H is divisible by the patch token size in ViT-4K (e.g. - 256 x 256).
		2. x then gets unfolded into a "batch" of [256 x 256] images.
		3. A pretrained ViT-256 model extracts the CLS token from each [256 x 256] image in the batch.
		4. These batch-of-features are then reshaped into a 2D feature grid (of width "w_256" and height "h_256".)
		5. This feature grid is then used as the input to ViT-4K, outputting [CLS]_4K.
		
		Args:
			- x (torch.Tensor): [1 x C x W' x H'] image tensor.
		
		Return:
			- features_cls4k (torch.Tensor): [1 x 192] cls token (d_4k = 192 by default).
		"""
		batch_256, w_256, h_256 = self.prepare_img_tensor(x)                    # 1. [1 x 3 x W x H] 
		batch_256 = batch_256.unfold(2, 256, 256).unfold(3, 256, 256)           # 2. [1 x 3 x w_256 x h_256 x 256 x 256] 
		batch_256 = rearrange(batch_256, 'b c p1 p2 w h -> (b p1 p2) c w h')    # 2. [B x 3 x 256 x 256], where B = (1*w_256*h_256)
		mini_bs_size = 2048								  
		features_cls256 = []
		for mini_bs in range(0, batch_256.shape[0], mini_bs_size):                       # 3. B may be too large for ViT-256. We further take minibatches of 256.
			minibatch_256 = batch_256[mini_bs:mini_bs+mini_bs_size].to(self.device256, non_blocking=True)
			features_cls256.append(self.model256(minibatch_256).detach().cpu()) # 3. Extracting ViT-256 features from [256 x 3 x 256 x 256] image batches.

		features_cls256 = torch.vstack(features_cls256)                         # 3. [B x 384], where 384 == dim of ViT-256 [ClS] token.
		features_cls256 = features_cls256.reshape(w_256, h_256, 384).transpose(0,1).transpose(0,2).unsqueeze(dim=0) 
		features_cls256 = features_cls256.to(self.device4k, non_blocking=True)  # 4. [1 x 384 x w_256 x h_256]
		features_cls4k = self.model4k.forward(features_cls256)                  # 5. [1 x 192], where 192 == dim of ViT-4K [ClS] token.
		return features_cls4k
	
	
	def forward_asset_dict(self, x: torch.Tensor):
		"""
		Forward pass of HIPT (given an image tensor x), with certain intermediate representations saved in 
		a dictionary (that is to be stored in a H5 file). See walkthrough of how the model works above.
		
		Args:
			- x (torch.Tensor): [1 x C x W' x H'] image tensor.
		
		Return:
			- asset_dict (dict): Dictionary of intermediate feature representations of HIPT and other metadata.
				- features_cls256 (np.array): [B x 384] extracted ViT-256 cls tokens
				- features_mean256 (np.array): [1 x 384] mean ViT-256 cls token (exluding non-tissue patches)
				- features_4k (np.array): [1 x 192] extracted ViT-4K cls token.
				- features_4k (np.array): [1 x 576] feature vector (concatenating mean ViT-256 + ViT-4K cls tokens)
	
		"""
		batch_256, w_256, h_256 = self.prepare_img_tensor(x)
		batch_256 = batch_256.unfold(2, 256, 256).unfold(3, 256, 256)
		batch_256 = rearrange(batch_256, 'b c p1 p2 w h -> (b p1 p2) c w h')
		
		features_cls256 = []
		for mini_bs in range(0, batch_256.shape[0], 256):
			minibatch_256 = batch_256[mini_bs:mini_bs+256].to(self.device256, non_blocking=True)
			features_cls256.append(self.model256(minibatch_256).detach().cpu())

		features_cls256 = torch.vstack(features_cls256)
		features_mean256 = features_cls256.mean(dim=0).unsqueeze(dim=0)

		features_grid256 = features_cls256.reshape(w_256, h_256, 384).transpose(0,1).transpose(0,2).unsqueeze(dim=0)
		features_grid256 = features_grid256.to(self.device4k, non_blocking=True)
		features_cls4k = self.model4k.forward(features_grid256).detach().cpu()
		features_mean256_cls4k = torch.cat([features_mean256, features_cls4k], dim=1)
		
		asset_dict = {
			'features_cls256': features_cls256.numpy(),
			'features_mean256': features_mean256.numpy(),
			'features_cls4k': features_cls4k.numpy(),
			'features_mean256_cls4k': features_mean256_cls4k.numpy()
		}
		return asset_dict


	def _get_region_attention_scores(self, region, scale=1):
		r"""
		Forward pass in hierarchical model with attention scores saved.
		
		Args:
		- region (PIL.Image):       4096 x 4096 Image 
		- model256 (torch.nn):      256-Level ViT 
		- model4k (torch.nn):       4096-Level ViT 
		- scale (int):              How much to scale the output image by (e.g. - scale=4 will resize images to be 1024 x 1024.)
		
		Returns:
		- np.array: [256, 256/scale, 256/scale, 3] np.array sequence of image patches from the 4K x 4K region.
		- attention_256 (torch.Tensor): [256, 256/scale, 256/scale, 3] torch.Tensor sequence of attention maps for 256-sized patches.
		- attention_4k (torch.Tensor): [1, 4096/scale, 4096/scale, 3] torch.Tensor sequence of attention maps for 4k-sized regions.
		"""
		eval_t = transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])
		x = eval_transforms()(region).unsqueeze(dim=0)

		batch_256, w_256, h_256 = self.prepare_img_tensor(x)
		batch_256 = batch_256.unfold(2, 256, 256).unfold(3, 256, 256)
		batch_256 = rearrange(batch_256, 'b c p1 p2 w h -> (b p1 p2) c w h')
		batch_256 = batch_256.to(self.device256, non_blocking=True)
		features_cls256 = self.model256(batch_256)

		attention_256 = self.model256.get_last_selfattention(batch_256)
		nh = attention_256.shape[1] # number of head
		attention_256 = attention_256[:, :, 0, 1:].reshape(256, nh, -1)
		attention_256 = attention_256.reshape(w_256*h_256, nh, 16, 16)
		attention_256 = nn.functional.interpolate(attention_256, scale_factor=int(16/scale), mode="nearest").cpu().numpy()

		features_grid256 = features_cls256.reshape(w_256, h_256, 384).transpose(0,1).transpose(0,2).unsqueeze(dim=0)
		features_grid256 = features_grid256.to(self.device4k, non_blocking=True)
		features_cls4k = self.model4k.forward(features_grid256).detach().cpu()

		attention_4k = self.model4k.get_last_selfattention(features_grid256)
		nh = attention_4k.shape[1] # number of head
		attention_4k = attention_4k[0, :, 0, 1:].reshape(nh, -1)
		attention_4k = attention_4k.reshape(nh, w_256, h_256)
		attention_4k = nn.functional.interpolate(attention_4k.unsqueeze(0), scale_factor=int(256/scale), mode="nearest")[0].cpu().numpy()

		if scale != 1:
			batch_256 = nn.functional.interpolate(batch_256, scale_factor=(1/scale), mode="nearest")

		return tensorbatch2im(batch_256), attention_256, attention_4k


	def prepare_img_tensor(self, img: torch.Tensor, patch_size=256):
		"""
		Helper function that takes a non-square image tensor, and takes a center crop s.t. the width / height
		are divisible by 256.
		
		(Note: "_256" for w / h is should technically be renamed as "_ps", but may not be easier to read.
		Until I need to make HIPT with patch_sizes != 256, keeping the naming convention as-is.)
		
		Args:
			- img (torch.Tensor): [1 x C x W' x H'] image tensor.
			- patch_size (int): Desired patch size to evenly subdivide the image.
		
		Return:
			- img_new (torch.Tensor): [1 x C x W x H] image tensor, where W and H are divisble by patch_size.
			- w_256 (int): # of [256 x 256] patches of img_new's width (e.g. - W/256)
			- h_256 (int): # of [256 x 256] patches of img_new's height (e.g. - H/256)
		"""
		make_divisble = lambda l, patch_size: (l - (l % patch_size))
		b, c, w, h = img.shape
		load_size = make_divisble(w, patch_size), make_divisble(h, patch_size)
		w_256, h_256 = w // patch_size, h // patch_size
		img_new = transforms.CenterCrop(load_size)(img)
		return img_new, w_256, h_256

In [10]:
class VisioMel(nn.Module):
    
    def __init__(self):
        
        super().__init__()
        
        pretrained_weights256 = '../src/hipt/Checkpoints/vit256_small_dino.pth'
        pretrained_weights4k = '../src/hipt/Checkpoints/vit4k_xs_dino.pth'
        
        device1 = torch.device('cuda:1')
        device2 = torch.device('cuda:1')
        
        ### ViT_256 + ViT_4K loaded into HIPT_4K API
        self.backbone = HIPT_4K(pretrained_weights256, pretrained_weights4k, device1, device2)
        #for param in self.backbone.parameters():
        #    param.requires_grad = False
        
        #self.backbone_wsi
        encoder_layer = nn.TransformerEncoderLayer(d_model=192, nhead=6)
        self.wsi_transformer = nn.TransformerEncoder(encoder_layer, num_layers=4)
        self.cls = nn.Parameter(torch.zeros(192))
        
        self.body = nn.Embedding(20, 11)
        self.sex = nn.Embedding(2, 3)
        self.mel = nn.Embedding(3, 5)
        
        self.act = nn.ReLU()
        
        self.cls1 = nn.Linear(212, 40)
        #self.cls2 = nn.Linear(100, 35)
        self.cls3 = nn.Linear(40, 10)
        self.cls4 = nn.Linear(10, 1)


    def forward(self, tiles, xage ,xsex, xbody, xmel):
        
        #with torch.no_grad():
        wsi_tokens = [self.cls]
        for i in range(0, tiles.shape[0]):
            tile = torch.select(tiles, 0 , i).unsqueeze(0)
            x = self.backbone(tile) #.forward(x)
            wsi_tokens.append(x)   

        x1 = torch.vstack(wsi_tokens)
        x1 = self.wsi_transformer(x1.unsqueeze(1))[0,:].squeeze(1)
        #x = x1
        x2 = self.sex(xsex)
        x3 = self.body(xbody)
        x4 = self.mel(xmel)  
        x = torch.cat([x1, xage, x2, x3, x4], dim=1)
        
        x = self.act(x)
        x = self.cls1(x)
        x = self.act(x)
        #x = self.cls2(x)
        #x = self.act(x)
        x = self.cls3(x)
        x = self.act(x)
        x = self.cls4(x)

        return x

In [11]:
device = torch.device('cuda:1')

#model = VisioMel()
#model = torch.nn.DataParallel(model, device_ids=[0, 1])
model = VisioMel().to(device)
#model = torch.compile(model)
model

# of Patches: 196


VisioMel(
  (backbone): HIPT_4K(
    (model256): VisionTransformer(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 384, kernel_size=(16, 16), stride=(16, 16))
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (blocks): ModuleList(
        (0-11): 12 x Block(
          (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
          (attn): Attention(
            (qkv): Linear(in_features=384, out_features=1152, bias=True)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=384, out_features=384, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
          )
          (drop_path): Identity()
          (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=384, out_features=1536, bias=True)
            (act): GELU(approximate='none')
            (fc2): Linear(in_features=1536, out_features=384, bias=True)
            (drop): Dropout(

In [12]:
epochs = 20000

#optimizer = torch.optim.SGD(model.parameters(), lr=5e-3)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4)
criterion = nn.BCEWithLogitsLoss()

In [13]:
ckpt = torch.load('../artifacts/ckpt_200_steps.pth')
model.load_state_dict(ckpt['model_state_dict'])
optimizer.load_state_dict(ckpt['optimizer_state_dict'])

In [14]:

for epoch_i in range(0, epochs):
    
    print(f'Epoch {epoch_i}')

    total_train_loss = 0
    model.train()

    optimizer.zero_grad()
    
    for step, batch in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):

        b_img = batch[0].squeeze(0) # xage ,xsex, xbody, xmel
        b_age = batch[1].unsqueeze(0).to(device)
        b_sex = batch[2].to(device)
        b_body = batch[3].to(device)
        b_mel = batch[4].to(device)
        b_labels = batch[5].to(device)

        with torch.cuda.amp.autocast(enabled=False):
            b_logits = model(b_img, b_age, b_sex, b_body, b_mel).squeeze(-1)
                
        loss = criterion(b_logits,b_labels)
        loss.backward()

        total_train_loss += loss.item()

        #torch.nn.utils.clip_grad_norm_(model.parameters(), 2.0)
        optimizer.step()
        optimizer.zero_grad()
        
        if step % 200 == 0:
            torch.save({
                'epoch': epochs,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                }, f'../artifacts/ckpt_200_steps.pth')
            
    
    avg_train_loss = (total_train_loss/len(train_dataloader))
    
    print(f'Train: loss {avg_train_loss:.5f}')

    model.eval()

    total_dev_loss = 0

    for step, batch in enumerate(valid_dataloader):

        b_img = batch[0].squeeze(0) # xage ,xsex, xbody, xmel
        b_age = batch[1].unsqueeze(0).to(device)
        b_sex = batch[2].to(device)
        b_body = batch[3].to(device)
        b_mel = batch[4].to(device)
        
        b_labels = batch[5].to(device)

        with torch.cuda.amp.autocast(enabled=False):
            with torch.no_grad(): 
                b_logits = model(b_img, b_age, b_sex, b_body, b_mel).squeeze(-1)

        loss = criterion(b_logits,b_labels)
        total_dev_loss += loss.item()
    
    avg_dev_loss = (total_dev_loss/len(valid_dataloader))

    print(f'Dev: loss {avg_dev_loss:.5f}')
    
    torch.save({
        'epoch': epochs,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        }, f'../artifacts/ckpt_{epoch_i}.pth')

Epoch 0


 17%|█▋        | 187/1073 [54:15<3:42:14, 15.05s/it]